In [60]:
import numpy as np
import matplotlib.pyplot as plt

In [61]:
with open('../headlinesData/Bad.csv') as bad:
    bad = bad.read().split('\n')
    

In [62]:
bad

['"FTC Sues Block $ 8.5 Billion Deal Combine Makers Coach , Michael Kors"',
 'Verizon Stock Falls After Revenue Misses Expectations',
 'Insurer Earnings Make Hospital Investors Sweat',
 '� Made Advertising � Websites Are Marketing Industry � Latest Messy Situation',
 'PwC Looks Shrink New U.S . Consulting Partner Class More Than 50 %',
 '"Companies Belly Up Cash Buffet , Five Charts"',
 'U.S . Fines Thai Company SCG Plastics $ 20 Million Over Alleged Iran Sanctions Violations',
 '"Real-Estate Agents , Investment Advisers Chafe New Anti-Money-Laundering Rules"',
 'Prologis Warns Slowing Industrial Real-Estate Market',
 '"CSX Revenue , Profit Fall Despite Rising Freight Volumes"',
 'Salesforce Calls AI Emissions Regulations Concerns Grow Over Tech Sector � Carbon Footprint',
 '"FTC Blocks Tapestry , Capri $ 8.5 Billion Merger ."',
 'GE Aerospace � Earnings Are Going Be Strange . Stay Tuned .',
 'Tesla Post Earnings After Tough Start Year',
 'Cathie Wood � Popular ARK Funds Are Sinking Fa

In [63]:
with open('../headlinesData/Good.csv') as good:
    good = good.read().split('\n')

In [64]:
with open('../headlinesData/Neutral.csv') as neutral:
    neutral = neutral.read().split('\n')

In [65]:
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()

good = v.fit_transform(good)

bad = v.transform(bad)

neutral = v.transform(neutral)

In [66]:
from sklearn import svm

clf = svm.SVC(kernel='linear')

In [67]:
trainGood = good[:int(good.shape[0]*.8)]
trainBad = bad[:int(bad.shape[0]*.8)]
trainNeutral = neutral[:int(neutral.shape[0]*.8)]

testGood = good[int(good.shape[0]*.8):]
testBad = bad[int(bad.shape[0]*.8):]
testNeutral = neutral[int(neutral.shape[0]*.8):]

<55x327 sparse matrix of type '<class 'numpy.float64'>'
	with 133 stored elements in Compressed Sparse Row format>

In [73]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, tol=1e-3)
def train():
    goodStep = int(trainGood.shape[0]/10)
    badStep = int(trainBad.shape[0]/10)
    neutralStep = int(trainNeutral.shape[0]/10)
    for i in range(9):
        goodSlice = trainGood[i*goodStep:(i+1)*goodStep].toarray()
        badSlice = trainBad[i*badStep:(i+1)*badStep].toarray()
        neutralSlice = trainNeutral[i*neutralStep:(i+1)*neutralStep].toarray()

        slices = [s for s in [goodSlice, badSlice, neutralSlice] if s.shape[0] > 0]
        if not slices:
            continue

        train_data = np.concatenate(slices)
        train_labels = np.concatenate((np.ones(goodSlice.shape[0]), np.zeros(badSlice.shape[0]), np.zeros(neutralSlice.shape[0])))
        clf.partial_fit(train_data, train_labels, classes=np.array([0, 1]))

In [75]:
train()

In [76]:
def test():
    goodStep = int(testGood.shape[0]/10)
    badStep = int(testBad.shape[0]/10)
    neutralStep = int(testNeutral.shape[0]/10)
    for i in range(9):
        goodSlice = testGood[i*goodStep:(i+1)*goodStep].toarray()
        badSlice = testBad[i*badStep:(i+1)*badStep].toarray()
        neutralSlice = testNeutral[i*neutralStep:(i+1)*neutralStep].toarray()

        slices = [s for s in [goodSlice, badSlice, neutralSlice] if s.shape[0] > 0]
        if not slices:
            continue

        test_data = np.concatenate(slices)
        test_labels = np.concatenate((np.ones(goodSlice.shape[0]), np.zeros(badSlice.shape[0]), np.zeros(neutralSlice.shape[0])))
        print(clf.score(test_data, test_labels))

In [77]:
test()

0.8571428571428571
0.7142857142857143
0.8571428571428571
0.8571428571428571
0.7142857142857143
0.7142857142857143
0.5714285714285714
0.8571428571428571
0.7142857142857143


In [ ]:
#TODO: maybe try adding a column for what the stock rise or fall was the day before? that would take a lot of work though, as data would have to be split up and given a date. maybe make two models, this current one and one that combines the news direction with the stock direction